# CDCTC Single Parent Work Requirement Reform Analysis

This notebook analyzes a reform to the Child and Dependent Care Tax Credit (CDCTC) that would make families eligible if **at least one parent works**, instead of the current law requiring **both parents to work**.

## Background

Under current law, the CDCTC caps eligible childcare expenses at the **minimum** of both spouses' earnings for married couples. This means if one spouse earns $50,000 and the other earns $0, the cap is $0 and the family receives no credit.

The reform changes this to use the **maximum** of both spouses' earnings, allowing families to qualify if at least one parent works.

In [ ]:
from policyengine_us import Microsimulation
from policyengine_us.reforms.cdcc.cdcc_single_parent_work_requirement import (
    cdcc_single_parent_work_requirement
)
import pandas as pd

## Microsimulation Analysis (2026)

In [ ]:
YEAR = 2026

# Baseline simulation
print("Running baseline simulation...")
baseline = Microsimulation()
baseline_cdcc = baseline.calculate('cdcc', YEAR)

# Reform simulation
print("Running reform simulation...")
reform = Microsimulation(reform=cdcc_single_parent_work_requirement)

# Clear cached downstream variables to ensure recalculation
# Note: This reform overrides cdcc_relevant_expenses directly 
for var_name in ['cdcc_relevant_expenses', 'cdcc_potential', 'cdcc', 'capped_cdcc']:
    holder = reform.get_holder(var_name)
    if holder:
        holder.delete_arrays()

reform_cdcc = reform.calculate('cdcc', YEAR)
print("Done.")

In [ ]:
# Calculate metrics
baseline_total = baseline_cdcc.sum()
reform_total = reform_cdcc.sum()
cost = reform_total - baseline_total

baseline_claimants = (baseline_cdcc > 0).sum()
reform_claimants = (reform_cdcc > 0).sum()
new_claimants = reform_claimants - baseline_claimants

baseline_avg = baseline_cdcc[baseline_cdcc > 0].mean()
reform_avg = reform_cdcc[reform_cdcc > 0].mean()

# New claimants average
new_claimant_mask = (baseline_cdcc.values == 0) & (reform_cdcc.values > 0)
new_claimants_avg = reform_cdcc[new_claimant_mask].mean() if new_claimant_mask.any() else 0

In [ ]:
# Create results table
results = pd.DataFrame({
    'Metric': ['Total Cost', 'Claimants', 'Average Credit', 'New Claimants Avg'],
    'Baseline': [
        f'${baseline_total/1e9:.2f}B',
        f'{baseline_claimants/1e6:.2f}M',
        f'${baseline_avg:.0f}',
        '—'
    ],
    'Reform': [
        f'${reform_total/1e9:.2f}B',
        f'{reform_claimants/1e6:.2f}M',
        f'${reform_avg:.0f}',
        f'${new_claimants_avg:.0f}'
    ],
    'Change': [
        f'+${cost/1e9:.2f}B ({cost/baseline_total*100:.1f}%)',
        f'+{new_claimants/1e6:.2f}M',
        f'${reform_avg - baseline_avg:+.0f}',
        '—'
    ]
})

results

## Summary

The CDCTC single parent work requirement reform would:

- **Increase federal CDCTC cost** by approximately $0.74 billion annually (+12.7%)
- **Increase state CDCC costs** by approximately $29 million (primarily South Carolina and Rhode Island)
- **Benefit approximately 920,000 additional families** who currently don't qualify because only one spouse works
- **Reduce the average credit slightly** (from $1,036 to $1,004) as newly eligible families tend to receive smaller credits

The reform primarily benefits one-earner married couples with childcare expenses who are currently excluded from the credit.

**Implementation note:** This reform overrides `cdcc_relevant_expenses` directly rather than `min_head_spouse_earned` to avoid affecting other programs that use the minimum earnings calculation for non-CDCC purposes.

## State CDCC Impacts

Some states offer child care credits that directly match or piggyback on the federal CDCC. These state credits will also increase under the reform since they reference the federal credit amount.

In [ ]:
# State CDCC variables that match/reference federal CDCC
state_cdcc_vars = [
    ('sc_cdcc', 'South Carolina'),
    ('ri_cdcc', 'Rhode Island'),
    ('ny_cdcc', 'New York'),
    ('mn_cdcc', 'Minnesota'),
    ('me_child_care_credit', 'Maine'),
    ('ma_dependent_care_credit', 'Massachusetts'),
    ('hi_cdcc', 'Hawaii'),
    ('co_cdcc', 'Colorado'),
    ('vt_cdcc', 'Vermont'),
]

state_results = []
for var_name, state_name in state_cdcc_vars:
    try:
        baseline_val = baseline.calculate(var_name, YEAR)
        reform_val = reform.calculate(var_name, YEAR)
        
        baseline_total = float(baseline_val.sum())
        reform_total = float(reform_val.sum())
        change = reform_total - baseline_total
        
        if baseline_total > 0 or change != 0:
            state_results.append({
                'State': state_name,
                'Baseline ($M)': round(baseline_total/1e6, 2),
                'Reform ($M)': round(reform_total/1e6, 2),
                'Change ($M)': round(change/1e6, 2),
                'Change (%)': round((change / baseline_total * 100), 1) if baseline_total > 0 else 0
            })
    except:
        pass

state_df = pd.DataFrame(state_results)
state_df = state_df[state_df['Change ($M)'] != 0].sort_values('Change ($M)', ascending=False)
state_df

In [ ]:
total_state_impact = state_df['Change ($M)'].sum()
print(f"Total State CDCC Impact: ${total_state_impact:.2f}M")
print(f"Combined Federal + State Impact: ${cost/1e6 + total_state_impact:.0f}M")

## Single Household Example

In [ ]:
from policyengine_us import Simulation

# Example: Married couple where only one spouse works
situation = {
    'people': {
        'adult1': {'age': {YEAR: 35}, 'employment_income': {YEAR: 50_000}},
        'adult2': {'age': {YEAR: 35}, 'employment_income': {YEAR: 0}},
        'child': {'age': {YEAR: 5}},
    },
    'tax_units': {
        'tax_unit': {
            'members': ['adult1', 'adult2', 'child'],
            'tax_unit_childcare_expenses': {YEAR: 5_000},
        }
    },
    'families': {'family': {'members': ['adult1', 'adult2', 'child']}},
    'households': {
        'household': {
            'members': ['adult1', 'adult2', 'child'],
            'state_code': {YEAR: 'CA'},
        }
    },
    'marital_units': {
        'marital_unit': {'members': ['adult1', 'adult2']},
    },
    'spm_units': {
        'spm_unit': {'members': ['adult1', 'adult2', 'child']},
    },
}

# Baseline
baseline_sim = Simulation(situation=situation)
baseline_credit = baseline_sim.calculate('cdcc', YEAR)[0]

# Reform
reform_sim = Simulation(situation=situation, reform=cdcc_single_parent_work_requirement)
reform_credit = reform_sim.calculate('cdcc', YEAR)[0]

print(f"Example: Married couple, one earner ($50k), one child, $5k childcare expenses")
print(f"  Baseline CDCC: ${baseline_credit:.0f}")
print(f"  Reform CDCC:   ${reform_credit:.0f}")
print(f"  Benefit:       ${reform_credit - baseline_credit:.0f}")